In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
-- Welcome to Snowflake Notebooks!
-- Try out a SQL cell to generate some data.
SELECT 'FRIDAY' as SNOWDAY, 0.2 as CHANCE_OF_SNOW
UNION ALL
SELECT 'SATURDAY',0.5
UNION ALL 
SELECT 'SUNDAY', 0.9;

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
my_df = cell2.to_pandas()

# Chart the data
st.subheader("Chance of SNOW ❄️")
st.line_chart(my_df, x='SNOWDAY', y='CHANCE_OF_SNOW')

# Give it a go!
st.subheader("Try it out yourself and show off your skills 🥇")

In [ ]:
CREATE ROLE dev_role;
CREATE USER dev PASSWORD = 'Connect!1it2026' DEFAULT_ROLE = dev_role;
GRANT USAGE ON WAREHOUSE Compute_WH TO ROLE dev_role;
GRANT USAGE ON DATABASE training TO ROLE dev_role;

In [ ]:
-- arn:aws:iam::529350069891:role/snowflake_s3_role
CREATE OR REPLACE STORAGE INTEGRATION s3_storage_int
  TYPE = EXTERNAL_STAGE
  STORAGE_PROVIDER = 'S3'
  ENABLED = TRUE
  STORAGE_AWS_ROLE_ARN = 'arn:aws:iam::529350069891:role/snowflake_s3_role'
  STORAGE_ALLOWED_LOCATIONS = ('s3://seiu-dev-b2b-datafeed/Inbound/raw/docebo_user/')
  COMMENT = 'Integration for Docebo Medallion Pipeline S3 buckets';

In [ ]:
-- arn:aws:iam::529350069891:role/snowflake_s3_role
CREATE OR REPLACE STORAGE INTEGRATION s3_alone_int
  TYPE = EXTERNAL_STAGE
  STORAGE_PROVIDER = 'S3'
  ENABLED = TRUE
  STORAGE_AWS_ROLE_ARN = 'arn:aws:iam::529350069891:role/snowflake_s3_role'
  STORAGE_ALLOWED_LOCATIONS = ('s3://sf-docebo-store/')
  COMMENT = 'Integration for Docebo User Pipeline S3 buckets';

In [ ]:
DESCRIBE STORAGE INTEGRATION s3_alone_int;

In [ ]:
-- 创建通知集成（这会在 Snowflake 账户中创建 SQS 队列）
CREATE NOTIFICATION INTEGRATION snowpipe_notification
  TYPE = QUEUE
  NOTIFICATION_PROVIDER = AWS_SQS
  ENABLED = TRUE
  DIRECTION = INBOUND
  AWS_SQS_ROLE_ARN = 'arn:aws:iam::529350069891:role/snowflake_s3_role'
  AWS_SQS_EXTERNAL_ID = 'SNOWFLAKE_EXTERNAL_ID';  -- Snowflake 会提供

-- 获取 SQS 队列 ARN 和 IAM 角色信息
DESC NOTIFICATION INTEGRATION snowpipe_notification;

In [ ]:
GRANT USAGE ON INTEGRATION s3_storage_int TO ROLE dev_role;

In [ ]:
CREATE OR REPLACE STAGE ingestion.dev_s3_inbound_stage
  URL = 's3://seiu-dev-b2b-datafeed/Inbound/raw/docebo_user/'
  STORAGE_INTEGRATION = s3_storage_int
  COMMENT = 'External stage for S3 data ingestion'
  FILE_FORMAT = (TYPE = CSV SKIP_HEADER = 0),
  DIRECTORY = ( ENABLE = true AUTO_REFRESH = true );

GRANT USAGE ON STAGE ingestion.dev_s3_inbound_stage TO ROLE dev_role;

In [ ]:
SHOW INTEGRATIONS;

In [ ]:
USE DATABASE training;
USE SCHEMA ingestion;
SHOW STAGES;

DESC STAGE DEV_S3_INBOUND_STAGE;

In [ ]:
CREATE OR REPLACE TABLE bronze.raw_docebo_user (
  ingestion_ts     TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),
  file_name        STRING,
  row_number       INTEGER,
  row_data         ARRAY,
  raw_line         STRING
);


In [ ]:
GRANT USAGE ON SCHEMA TRAINING.BRONZE TO ROLE dev_role;

In [ ]:
LIST @ingestion.dev_s3_inbound_stage;

In [ ]:
select $1, $2, $3, $4, $5 from @ingestion.dev_s3_inbound_stage/dd0_20260103171849.csv;

In [ ]:
COPY INTO bronze.raw_docebo_user (
  file_name,
  row_number,
  row_data,
  raw_line
)
FROM (
  SELECT
    METADATA$FILENAME,
    METADATA$FILE_ROW_NUMBER,
    ARRAY_CONSTRUCT($1, $2, $3, $4, $5, $6),
    $1
  FROM @ingestion.dev_s3_inbound_stage
)
FILE_FORMAT = (
  TYPE = CSV
  SKIP_HEADER = 0
)
ON_ERROR = CONTINUE;

In [ ]:
select * from bronze.raw_docebo_user;

In [ ]:
CREATE OR REPLACE PIPE ingestion.dev_s3_inbound_pipe
  AUTO_INGEST = TRUE
AS
COPY INTO bronze.raw_docebo_user (
  file_name,
  row_number,
  row_data,
  raw_line
)
FROM (
  SELECT
    METADATA$FILENAME,
    METADATA$FILE_ROW_NUMBER,
    ARRAY_CONSTRUCT($1, $2, $3, $4, $5, $6),
    $1
  FROM @ingestion.sandbox_docebo_user_inbound
)
FILE_FORMAT = (
  TYPE = CSV
  SKIP_HEADER = 0
)
ON_ERROR = CONTINUE;

In [ ]:
CREATE OR REPLACE STREAM bronze.raw_docebo_user_stream
ON TABLE bronze.raw_docebo_user
APPEND_ONLY = TRUE;

In [ ]:
-- 检查流的状态
SHOW STREAMS IN SCHEMA BRONZE;

SELECT 
    "name" as stream_name,
    "database_name",
    "schema_name",
    "table_name" as source_table,
    "owner",
    "comment",
    "created_on"
FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()));
-- 查看流的详细信息，包括其 schema
-- SELECT 
--     STREAM_CATALOG as database_name,
--     STREAM_SCHEMA as schema_name,
--     STREAM_NAME as stream_name,
--     TABLE_NAME as source_table
-- FROM INFORMATION_SCHEMA.STREAMS
-- WHERE STREAM_NAME = 'RAW_DOCEBO_USER_STREAM'
--   AND STREAM_SCHEMA = 'BRONZE';

-- 检查流的内容
-- SELECT SYSTEM$STREAM_HAS_DATA('BRONZE.RAW_DOCEBO_USER_STREAM');
-- 返回 TRUE = 有数据，FALSE = 无数据，NULL = 流无效

In [ ]:
TRUNCATE TABLE bronze.raw_docebo_user;

In [ ]:
SELECT 
    COUNT(*) as total_rows_in_stream,
    LISTAGG(DISTINCT METADATA$ACTION, ', ') as actions_present
FROM BRONZE.RAW_DOCEBO_USER_STREAM;

-- 方法3: 查看流的DDL
-- SELECT GET_DDL('STREAM', 'BRONZE.RAW_DOCEBO_USER_STREAM');

In [ ]:
SELECT ROW_DATA[0], ROW_DATA[1],ROW_DATA[2], ROW_DATA[3], ROW_DATA[4]
FROM BRONZE.RAW_DOCEBO_USER_STREAM
WHERE row_number > 1;

SELECT *
FROM BRONZE.RAW_DOCEBO_USER_STREAM
WHERE row_number > 1;

In [ ]:
-- select * from bronze.raw_docebo_user;
USE DATABASE training;
USE SCHEMA ingestion;

SHOW PIPES LIKE 'DEV_S3_INBOUND_PIPE';
SELECT
  *
FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()));

DESC PIPE DEV_S3_INBOUND_PIPE;

SELECT SYSTEM$PIPE_STATUS('DEV_S3_INBOUND_PIPE');

SELECT
  FILE_NAME,
  LAST_LOAD_TIME,
  ROW_COUNT,
  ERROR_COUNT,
  STATUS
FROM TABLE(
  INFORMATION_SCHEMA.COPY_HISTORY(
    TABLE_NAME => 'bronze.raw_docebo_user',
    START_TIME => DATEADD(hour, -24, CURRENT_TIMESTAMP())
  )
)
-- WHERE FILE_NAME ilike '%dd0_20260103171849.csv%'
ORDER BY LAST_LOAD_TIME DESC;

-- SELECT *
-- FROM INFORMATION_SCHEMA.LOAD_HISTORY
-- WHERE DATE(LAST_LOAD_TIME) >= CURRENT_DATE() - 1
-- ORDER BY LAST_LOAD_TIME DESC;

In [ ]:
SELECT SYSTEM$PIPE_STATUS('DEV_S3_INBOUND_PIPE');

In [ ]:
SELECT PARSE_JSON(SYSTEM$PIPE_STATUS('DEV_S3_INBOUND_PIPE'));

In [ ]:
-- 查看管道定义
DESC PIPE DEV_S3_INBOUND_PIPE;

-- 或者
SHOW PIPES LIKE '%DEV_S3_INBOUND_PIPE%';

In [ ]:
SHOW PIPES LIKE '%DEV_S3_INBOUND_PIPE%';
-- 从管道定义中获取信息
SELECT 
  "definition" as pipe_definition,
  "notification_channel" as channel_name
FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()))
WHERE "name" = 'DEV_S3_INBOUND_PIPE';

In [ ]:
ALTER PIPE DEV_S3_INBOUND_PIPE REFRESH;

In [ ]:
-- 查看管道详细信息
DESC PIPE DEV_S3_INBOUND_PIPE;

-- 或使用 SHOW PIPES
SHOW PIPES LIKE 'DEV_S3_INBOUND_PIPE';

-- 检查管道定义
SELECT 
    pipe_name,
    definition,
    is_autoingest_enabled,
    notification_channel_name
FROM information_schema.pipes
WHERE pipe_name = 'DEV_S3_INBOUND_PIPE';

In [ ]:
ALTER PIPE ingestion.dev_s3_inbound_pipe SET PIPE_EXECUTION_PAUSED = FALSE;

In [ ]:
SELECT SYSTEM$PIPE_FORCE_RESUME('INGESTION.DEV_S3_INBOUND_PIPE');

In [ ]:
-- 需要 ACCOUNTADMIN 或 MONITORING 权限
USE ROLE ACCOUNTADMIN;

-- 查看管道使用历史
SELECT *
FROM SNOWFLAKE.ACCOUNT_USAGE.PIPE_USAGE_HISTORY
WHERE PIPE_NAME = 'DEV_S3_INBOUND_PIPE'
  AND START_TIME >= DATEADD(hour, -24, CURRENT_TIMESTAMP())
ORDER BY START_TIME DESC;



In [ ]:
SELECT SYSTEM$PIPE_STATUS('INGESTION.DEV_S3_INBOUND_PIPE');


In [ ]:
-- 检查是否有未处理的文件
SELECT 
  SYSTEM$PIPE_STATUS('INGESTION.DEV_S3_INBOUND_PIPE') as pipe_status;

-- -- 查看管道负载历史
SELECT *
FROM TABLE(INFORMATION_SCHEMA.PIPE_USAGE_HISTORY(
  DATE_RANGE_START => DATEADD(hour, -2, CURRENT_TIMESTAMP())
));

In [ ]:
select * from INFORMATION_SCHEMA.STAGES;

In [ ]:
USE DATABASE TRAINING;
-- 简化的用户表
CREATE OR REPLACE TABLE silver.docebo_user (
    user_id VARCHAR(100),
    email VARCHAR(255),
    first_name VARCHAR(100),
    last_name VARCHAR(100),
    status VARCHAR(50),
    created_at TIMESTAMP_NTZ,
    source_file VARCHAR(500)
);

CREATE OR REPLACE TABLE silver.docebo_user_quarantine (
    -- 你要求的字段
    error_reason VARCHAR(500) NOT NULL,
    file_name VARCHAR(500) NOT NULL,
    row_number NUMBER NOT NULL,
    raw_row VARIANT,
    raw_line TEXT
);

In [ ]:
CREATE OR REPLACE TASK silver.load_docebo_user_task
WAREHOUSE = COMPUTE_WH
SCHEDULE = 'USING CRON */5 * * * * UTC'
WHEN SYSTEM$STREAM_HAS_DATA('bronze.raw_docebo_user_stream')
AS
BEGIN
    -- Good rows
    INSERT INTO silver.docebo_user (
        user_id,
        email,
        first_name,
        last_name,
        status,
        created_at,
        source_file
    )
    SELECT
        row_data[0]::STRING,
        row_data[1]::STRING,
        row_data[2]::STRING,
        row_data[3]::STRING,
        row_data[4]::STRING,
        TRY_TO_TIMESTAMP_NTZ(row_data[5]::STRING),
        file_name
    FROM bronze.raw_docebo_user_stream
    WHERE METADATA$ACTION = 'INSERT'
      AND row_data IS NOT NULL
      AND ARRAY_SIZE(row_data) >= 6;

    -- Bad rows
    INSERT INTO silver.docebo_user_quarantine (
        error_reason,
        file_name,
        row_number,
        raw_row,
        raw_line
    )
    SELECT
        CASE
            WHEN row_data IS NULL THEN 'ROW_DATA_NULL'
            WHEN ARRAY_SIZE(row_data) < 6 THEN 'INSUFFICIENT_COLUMNS'
            WHEN TRY_TO_TIMESTAMP_NTZ(row_data[5]::STRING) IS NULL THEN 'INVALID_TIMESTAMP'
            ELSE 'UNKNOWN_ERROR'
        END,
        file_name,
        row_number,
        row_data,
        raw_line
    FROM bronze.raw_docebo_user_stream
    WHERE METADATA$ACTION = 'INSERT'
      AND (
            row_data IS NULL
         OR ARRAY_SIZE(row_data) < 6
      );
END;


In [ ]:
-- Good rows
    INSERT INTO silver.docebo_user (
        user_id,
        email,
        first_name,
        last_name,
        status,
        created_at,
        source_file
    )
    SELECT
        row_data[0]::STRING,
        row_data[1]::STRING,
        row_data[2]::STRING,
        row_data[3]::STRING,
        row_data[4]::STRING,
        TRY_TO_TIMESTAMP_NTZ(row_data[5]::STRING),
        file_name
    FROM bronze.raw_docebo_user_stream;
    WHERE METADATA$ACTION = 'INSERT'
      AND row_data IS NOT NULL
      AND ARRAY_SIZE(row_data) >= 6
      AND TRY_TO_TIMESTAMP_NTZ(row_data[5]::STRING) IS NOT NULL;

    -- Bad rows
    INSERT INTO silver.docebo_user_quarantine (
        error_reason,
        file_name,
        row_number,
        raw_row,
        raw_line
    )
    SELECT
        CASE
            WHEN row_data IS NULL THEN 'ROW_DATA_NULL'
            WHEN ARRAY_SIZE(row_data) < 6 THEN 'INSUFFICIENT_COLUMNS'
            WHEN TRY_TO_TIMESTAMP_NTZ(row_data[5]::STRING) IS NULL THEN 'INVALID_TIMESTAMP'
            ELSE 'UNKNOWN_ERROR'
        END,
        file_name,
        row_number,
        row_data,
        raw_line
    FROM bronze.raw_docebo_user_stream;
    -- WHERE METADATA$ACTION = 'INSERT'
    --   AND (
    --         row_data IS NULL
    --      OR ARRAY_SIZE(row_data) < 6
    --      OR TRY_TO_TIMESTAMP_NTZ(row_data[5]::STRING) IS NULL
    --   );

In [ ]:
EXECUTE TASK silver.load_docebo_user_task;

-- 等待几秒，然后检查执行结果
SELECT *
FROM TABLE(INFORMATION_SCHEMA.TASK_HISTORY())
WHERE NAME = 'LOAD_DOCEBO_USER_TASK'
ORDER BY SCHEDULED_TIME DESC
LIMIT 2;

-- 检查是否插入了数据
-- SELECT COUNT(*) as good_rows FROM silver.docebo_user;
-- SELECT COUNT(*) as bad_rows FROM silver.docebo_user_quarantine;
-- SELECT COUNT(*) as total_stream_rows FROM bronze.raw_docebo_user_stream;

In [ ]:
-- 暂停 TASK
-- ALTER TASK silver.load_docebo_user_task SUSPEND;

-- -- 恢复 TASK
-- ALTER TASK silver.load_docebo_user_task RESUME;

-- -- 查看 TASK 调度信息
DESC TASK silver.load_docebo_user_task;

-- 查看所有 TASK
SHOW TASKS IN ACCOUNT;

-- 查看 TASK 执行历史
SELECT *
FROM TABLE(INFORMATION_SCHEMA.TASK_HISTORY(
  SCHEDULED_TIME_RANGE_START => DATEADD('hour', -24, CURRENT_TIMESTAMP()),
  RESULT_LIMIT => 100
))
ORDER BY SCHEDULED_TIME DESC;

-- 查看特定 TASK 的执行历史
SELECT *
FROM TABLE(INFORMATION_SCHEMA.TASK_HISTORY(
  TASK_NAME => 'load_docebo_user_task',
  SCHEDULED_TIME_RANGE_START => DATEADD('day', -7, CURRENT_TIMESTAMP())
));

In [ ]:
select * from silver.docebo_user;

In [ ]:
SELECT * FROM SNOWFLAKE.ACCOUNT_USAGE.DATABASES
WHERE DATABASE_NAME LIKE 'USER$%';